---

# Model deployment

## Importar librerias

In [73]:
import mlflow
import sys
import os

import numpy as np
import pandas as pd
from mlflow.tracking import MlflowClient

sys.path.append("../../")
from ML.utils.mlflow_flow import set_tracking
from ML.utils.utils_yose import load_data, make_features
from dotenv import load_dotenv

load_dotenv()
ENDPOINT_URL = os.getenv("ENDPOINT_URL")
DAGSHUB_PUBLIC_ACCESS_KEY = os.getenv("MLFLOW_TRACKING_TOKEN")
MODEL_NAME = os.getenv("MODEL_NAME")
ALIAS = os.getenv("MODEL_ALIAS")

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/yosesotomayor/retoCasas_v2.mlflow"
os.environ["MLFLOW_REGISTRY_URI"] = "https://dagshub.com/yosesotomayor/retoCasas_v2.mlflow"



---

In [80]:
set_tracking(ENDPOINT_URL)
client = MlflowClient()
print("\n=== Model Registry ===")
try:
    rms = client.search_registered_models(max_results=5)
    print("OK, models:", [m.name for m in rms])
except Exception as e:
    print("Registry falla:", type(e).__name__, str(e)[:200])

try:
    m = mlflow.pyfunc.load_model("runs:/c5d7f7da87664b67ad1595f33557c4cc/model")
    #m = mlflow.pyfunc.load_model(f"models:/{MODEL_NAME}@challenger")
    print("\nCarga del modelo:/ --> OK")
except Exception as e:
    print("\nCarga del modelo:/ falla:", type(e).__name__, str(e)[:200])


=== Model Registry ===
OK, models: ['elnet_lgbm', 'neural_network_ymsc']



Carga del modelo:/ --> OK


---

In [81]:
sub_dir = "../../data/housing_data/"
_, df_test = load_data(sub_dir)

X_test = make_features(df_test.drop(["Id"], axis=1))

preds_orig = np.expm1(m.predict(X_test)) 
sub_dir = "../../data/housing_data_submissions/elnet_lgbm/"
os.makedirs(sub_dir, exist_ok=True)
submission_path = os.path.join(sub_dir, "submission_elnet_lgbm-kaggle.csv")

df_sub = pd.DataFrame({"Id": df_test["Id"], "SalePrice": preds_orig})
df_sub.to_csv(submission_path, index=False)